## Import des librairies ##

In [ ]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Bibliothèques pour Cross validation

#Affichage de Graphes
from IPython.display import Markdown, display, HTML

## 1. Lecture des données ##

1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

In [ ]:
#Lecture du fichier passagers avec label "Survived", pour le training
train = pd.read_csv('Data/passagers.csv')
train.sample(10)

1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples. 

In [ ]:
#Lecture du fichier Test.csv sans label "Survived", à calculer et envoyer à Kaggle
test  = pd.read_csv('Data/test.csv' , header = 0, dtype={'Age': np.float64})
test.sample(10)

1.3 Afficher les informations des deux DataFrames train et Test.

In [ ]:
# Infos sur train et test
display(Markdown('<span style="color:blue;font-weight:bold">train :</span>'))
train.info()
display(Markdown('<span style="color:blue;font-weight:bold">test :</span>'))
test.info()

1.4 Calculer le taux global de survie

In [ ]:
'%.2f%% de survie'%(train.Survived.mean()*100)

#### Question 1 : Quel Travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?




#### Question 2 : Quelle différence y-a-t'il entre les données train et les données test ? 




#### Question 3 : Quel est le rôle de ces deux sets de données ?




## 2. Analyse des features

2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet de la classe</span>'))
train[['Pclass', 'Survived']].groupby(['Pclass']).mean()

2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par classe

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Genre</span>'))
print (train[["Sex", "Survived"]].groupby(['Sex']).mean())

2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

In [ ]:
#On crée une colonne pour la taille de la famille
for dataset in [train,test]:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    display(HTML(dataset[['PassengerId','Name','Sex','Age','FamilySize']].sample(10).to_html()))
    print('-------------------------------------')

2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet de la taille de la famille</span>'))
print (train[['FamilySize', 'Survived']].groupby(['FamilySize']).count())
print (train[['FamilySize', 'Survived']].groupby(['FamilySize']).mean())

2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

In [ ]:
for dataset in [train,test]:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    display(HTML(dataset[['PassengerId','Name','Sex','Age','IsAlone']].sample(10).to_html()))

2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

In [ ]:
display(Markdown("<span style='color:blue;font-weight:bold'>Effet de la présence d'une famille</span>"))
print (train[['IsAlone', 'Survived']].groupby(['IsAlone']).count())
print (train[['IsAlone', 'Survived']].groupby(['IsAlone']).mean())

### Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?



### Question 2 : Même question pour la variable Sex 


### Question 3 : Comment traiter le variables liées à la famille du passager ?





## 3. Traitement des données manquantes (à faire sur les données Train et Test)

3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

In [ ]:
for dataset in [train,test]:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

3.2 Afficher l'influence du port d'embarquement sur la Survie 

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Port d\'embarquement</span>'))
print (train[['Embarked', 'Survived']].groupby(['Embarked']).mean())

3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

In [ ]:
for dataset in [train,test]:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales. 

In [ ]:
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
train.sample(5)

3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Prix du Ticket</span>'))
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare']).mean())

3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).

In [ ]:
for dataset in  [train,test]:
    age_null_count = dataset['Age'].isnull().sum()
    print("%i valeurs d'age manquent."%age_null_count)
    age_avg   = dataset['Age'].mean()
    age_std   = dataset['Age'].std()    
    # Création et attribution pour chaque valeur nulle d'une valeur entière aléatoire
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)    
    dataset.loc[np.isnan(dataset['Age']),'Age'] = age_null_random_list
    display(HTML(dataset[['PassengerId','Name','Sex','Age']].sample(10).to_html()))
    print('-------------------------------------')
    #dataset['Age'] = dataset['Age'].astype(int)

3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 catégories égales. 

In [ ]:
train['CategoricalAge'] = pd.cut(train['Age'], 5)
train.sample(10)

3.8 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [ ]:
display(Markdown('<span style="color:blue;font-weight:bold">Effet de l\'age</span>'))
train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge']).mean()

### Question 1 : Classer les variables par ordre d'intérêt

## 4 : Traitement de features complexes (Optionnel) 

4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

In [ ]:
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # Si le titre existe, on le renvoie
    if title_search:
        return title_search.group(1)
    return ""

for dataset in [train,test]:
    dataset['Title'] = dataset['Name'].apply(get_title)
    display(HTML(dataset[['PassengerId','Name','Sex','Title']].sample(10).to_html()))
    print('-------------------------------------')

4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurences par titres (en ligne) et par sexe (en colonne)

In [ ]:
pd.crosstab(train['Title'], train['Sex'])

4.3 : 
Remplacer 'Melle et 'Ms' par 'Miss'
Remplacer 'Mme' par 'Mrs'
Creer une valeur "Rare" pour les autres titres avec moins de 10 occurences. 

In [ ]:
for dataset in [train,test]:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                   'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
display(Markdown('<span style="color:blue;font-weight:bold">Effet du Titre</span>'))
print (train[['Title', 'Survived']].groupby(['Title']).mean())

## 5. Mise en forme des données non numériques

5.1 Attribuer une valeur numérique au Genre. Par exemple 0 pour 'female' et 1 pour 'male'

In [ ]:
for dataset in [train,test]:
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} )
    display(HTML(dataset[['PassengerId','Name','Sex']].sample(10).to_html()))

5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare'). 

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in [train,test]:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    display(HTML(dataset[['PassengerId','Name','Title']].sample(10).to_html()))

5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier.

In [ ]:
 for dataset in [train,test]:
    dataset['Title'] = dataset['Title'].fillna(0).astype(int)
    display(HTML(dataset[['PassengerId','Name','Title']].sample(10).to_html()))    

5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare (question 3.6)

In [ ]:
for dataset in [train,test]:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']= 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    display(HTML(dataset[['PassengerId','Name','Fare']].sample(10).to_html()))

5.5 Attribuer une valeur numérique aux 5 catégories de la variable Age (question 3.7)

In [ ]:
for dataset in [train,test]:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4
    dataset['Age'] = dataset['Age'].astype(int)
    display(HTML(dataset[['PassengerId','Name','Age']].sample(10).to_html()))

5.6 Attibuer une valeur numérique au Port d'embarquement (Embarked)

In [ ]:
 for dataset in [train,test]:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    display(HTML(dataset[['PassengerId','Name','Embarked']].sample(10).to_html()))

5.7 Supprimer les features inutiles

In [ ]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)    

5.8 Afficher les infos pour les deux datasets

In [ ]:
# Infos sur train et test
display(Markdown('<span style="color:blue;font-weight:bold">train mis en forme :</span>'))
train.info()
display(Markdown('<span style="color:blue;font-weight:bold">test mis en forme :</span>'))
test.info()

# Sauver les deux datasets dans des fichier csv. Eviter d'écrire un index (n° de ligne)

In [ ]:
train.to_csv('Data/train_clean.csv',index=False)
test.to_csv('Data/test_clean.csv',index=False)

##### Question : Sur combien de questions avez-vous utilisé les conseils ?
